In [1]:
import sys, os

sys.path.append(os.path.abspath('..'))

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = [8.3, 5.1]

In [3]:
from utils import load_dataset
from metrics import TrainingMetrics

In [4]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

In [5]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(2, 24, kernel_size=3)
        self.conv2 = nn.Conv2d(24, 49, kernel_size=3)
        
        # fully connected layers
        self.fc1 = nn.Linear(196, 128)
        self.fc2 = nn.Linear(128, 20)
        self.fc3 = nn.Linear(20, 10)
        self.classifier = nn.Linear(10, 1)
        
        # Regularizers
        self.drop = nn.Dropout(0.2)
        self.pool = nn.MaxPool2d(2,2)
        
        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        
        x = self.relu(self.fc1(x.flatten(start_dim=1)))
        x = self.drop(x)
        
        x = self.relu(self.fc2(x))
        x = self.drop(x)
        
        x = self.relu(self.fc3(x.flatten(start_dim=1)))
        
        x = self.sigmoid(self.classifier(x))
        
        return x.squeeze(), None

In [13]:
train_loader, test_loader = load_dataset()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw


  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw
Processing...


/Users/arnauddhaene/miniconda3/envs/ds/lib/python3.8/site-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-ggc7m97o/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [7]:
model = Net(200)

In [8]:
def accuracy(model, load):
    
    accuracy = 0.
    counter = 0
    
    model.eval()
    
    with torch.no_grad():
            for (input, target, _) in load:
                output, _ = model(input)
                
                accuracy += (output >= 0.5) == target
                counter += target.size(0)
                
    return (accuracy.sum() / counter).float().item()

In [9]:
criterion = nn.BCELoss()
eta = 1e-2
epochs = 25
decay = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=eta, weight_decay=decay)

metrics = TrainingMetrics()

for epoch in range(epochs):
    
    acc_loss = 0.
    
    model.train()
    
    for input, target, classes in train_loader:
        
        output, aux = model(input)
        loss = criterion(output, target.float())
        
        acc_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    metrics.add_entry(epoch, acc_loss, accuracy(model, train_loader))
    print(metrics)

Epoch 00 	Loss 014.007 	Accuracy 55.100
Epoch 01 	Loss 013.718 	Accuracy 56.000
Epoch 02 	Loss 012.425 	Accuracy 72.300
Epoch 03 	Loss 011.220 	Accuracy 79.800
Epoch 04 	Loss 009.231 	Accuracy 83.700
Epoch 05 	Loss 008.534 	Accuracy 83.600
Epoch 06 	Loss 008.343 	Accuracy 88.900
Epoch 07 	Loss 007.369 	Accuracy 84.700
Epoch 08 	Loss 006.559 	Accuracy 91.600
Epoch 09 	Loss 005.733 	Accuracy 91.400
Epoch 10 	Loss 005.106 	Accuracy 93.200
Epoch 11 	Loss 003.617 	Accuracy 92.600
Epoch 12 	Loss 004.558 	Accuracy 93.400
Epoch 13 	Loss 003.086 	Accuracy 94.500
Epoch 14 	Loss 002.869 	Accuracy 94.700
Epoch 15 	Loss 003.410 	Accuracy 95.200
Epoch 16 	Loss 002.988 	Accuracy 95.900
Epoch 17 	Loss 002.518 	Accuracy 97.100
Epoch 18 	Loss 002.114 	Accuracy 97.900
Epoch 19 	Loss 001.612 	Accuracy 97.900
Epoch 20 	Loss 002.555 	Accuracy 97.100
Epoch 21 	Loss 001.763 	Accuracy 97.700
Epoch 22 	Loss 001.941 	Accuracy 93.600
Epoch 23 	Loss 002.417 	Accuracy 96.900
Epoch 24 	Loss 001.228 	Accuracy 98.600


In [10]:
print(f"{4.555:06.3f}")

04.555


In [11]:
import pandas as pd
import seaborn as sns

training_stats = pd.DataFrame.from_dict(stats, orient='index')
training_stats['epoch'] = training_stats.index

NameError: name 'stats' is not defined

In [ ]:
ax_loss = sns.lineplot(data=training_stats, x="epoch", y="loss", label='loss')

ax_acc = ax_loss.twinx()

sns.lineplot(data=training_stats, x="epoch", y="accuracy", label='accuracy', ax=ax_acc, color='r')

plt.show()